# Coverage of fragments in STARR-seq results (Input 20x)

this is also a test for a newer code, including
- gzip the results file
- after updating the path names

**Set environment**

In [2]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



## Test: get the input files

**Input**

In [5]:
FD_BEDS=($(ls -d ${FD_RES}/data/Input?_20x/))
for FD_BED in ${FD_BEDS[@]}; do
    echo ${FD_BED}
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/


In [10]:
CHROM=chr17
FD_GEN=${FD_ANN}/genome/hg38
FN_GEN=hg38.chrom.sizes.${CHROM}

ls ${FD_GEN}
echo +++++++++++++++++++++++++++
ls ${FD_GEN}/${FN_GEN}

fasta                   hg38.chrom.sizes.chr16  hg38.chrom.sizes.chr4
hg38.chrom.sizes        hg38.chrom.sizes.chr17  hg38.chrom.sizes.chr5
hg38.chrom.sizes.chr1   hg38.chrom.sizes.chr18  hg38.chrom.sizes.chr6
hg38.chrom.sizes.chr10  hg38.chrom.sizes.chr19  hg38.chrom.sizes.chr7
hg38.chrom.sizes.chr11  hg38.chrom.sizes.chr2   hg38.chrom.sizes.chr8
hg38.chrom.sizes.chr12  hg38.chrom.sizes.chr20  hg38.chrom.sizes.chr9
hg38.chrom.sizes.chr13  hg38.chrom.sizes.chr21  hg38.chrom.sizes.chrX
hg38.chrom.sizes.chr14  hg38.chrom.sizes.chr22  hg38.chrom.sizes.chrY
hg38.chrom.sizes.chr15  hg38.chrom.sizes.chr3   hg38.chrom.sizes.rm
+++++++++++++++++++++++++++
/gpfs/fs1/data/reddylab/Kuei/annotation/genome/hg38/hg38.chrom.sizes.chr17


**Output**

In [11]:
ls ${FD_RES}/coverage

Input       Input2_20x  Input4_20x   TFX2_Dex   TFX4_Dex   TFX_Dex
Input1      Input3      Input5       TFX2_DMSO  TFX4_DMSO  TFX_DMSO
Input1_20x  Input3_20x  Input5_20x   TFX3_Dex   TFX5_Dex
Input2      Input4      target_PER1  TFX3_DMSO  TFX5_DMSO


In [ ]:
${FD_LOG}/coverage_perbase_chrom_input20x.txt

## Get coverage of chromosome chr17

In [16]:
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --mem 8G \
    -o ${FD_LOG}/coverage_perbase_chrom_input20x.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr17
TARGET=${CHROM}

### set input and output
#FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
FD_BEDS=($(ls -d ${FD_RES}/data/Input?_20x/))
FN_BED=${CHROM}.bed

FD_GEN=${FD_ANN}/genome/hg38
FN_GEN=hg38.chrom.sizes.${CHROM}

### START Message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/coverage/${SAMPLE}
    FN_OUT=${TARGET}_perbase.tsv.gz
    
    ### Loop START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 5 ${FD_BED}/${FN_BED}
    echo
    echo "Show the last few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    tail -n 5 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### run: Bedtools genomecov -d to get coverage per base
    echo "Get coverage per base"
    bedtools genomecov -i ${FD_BED}/${FN_BED} -g ${FD_GEN}/${FN_GEN} -d | gzip -c > ${FD_OUT}/${FN_OUT}
    echo
    
    ### Loop END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    zcat ${FD_OUT}/${FN_OUT} | head -n 3
    echo
    echo "Show the last few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    zcat ${FD_OUT}/${FN_OUT} | tail -n 3
    echo

done

### END message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27283781


In [ ]:
cat ${FD_LOG}/coverage_perbase_chrom_input20x.txt

Slurm Array Index: 
Time Stamp:         02-06-22+13:15:22

RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/coverage/Input1_20x/chr17_perbase.tsv.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
chr17	107410	108464
chr17	159026	160040
chr17	159426	160303
chr17	159510	160362
chr17	159977	160849

Show the last few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
chr17	83242601	83243570
chr17	83242601	83243555
chr17	83244670	83245775
chr17	83244681	83245751
chr17	83244716	83245595

++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x//chr17.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combe